In [72]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json
from datetime import datetime
import logging
from pathlib import Path 
import concurrent.futures
import tqdm
import yfinance as yf
import pandas as pd
import os
from os.path import join
from datetime import datetime
from pathlib import Path 

In [73]:
# Read the csv from stagging
path = '../../data/intermediate_02/financial-sheets.csv'
ticker_file_all = pd.read_csv(path)
ticker = 'ITRN'

In [74]:
# Get the company data from yfinance
stock_info = yf.Ticker(ticker)
hist = stock_info.history(period="max", interval = "1mo")

# Transform data to add growth info
hist = hist.reset_index()
hist['Date'] = pd.to_datetime(hist['Date'])
hist['year'] = hist['Date'].dt.year
hist.dropna(inplace=True)

In [75]:
ticker_file = ticker_file_all[ticker_file_all['ticker'] == ticker]

In [76]:
# Convert 'Date' column in 'hist' dataframe to string format
hist['Date'] = hist['Date'].astype(str)
# Extract the 'YYYY-MM' format from 'Date' column in 'ticker_file' dataframe
ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)
# Filter 'hist' dataframe based on 'YYYY-MM' values from 'ticker_file'
filtered_hist = hist[hist['Date'].str.slice(0, 7).isin(ticker_file['YearMonth'])]

C:\Users\germa\AppData\Local\Temp/ipykernel_6404/3425731416.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_file['YearMonth'] = ticker_file['Date'].str.slice(0, 7)


In [77]:
ticker_file['YearMonth']

26251    2009-12
26252    2010-12
26253    2011-12
26254    2012-12
26255    2013-12
26256    2014-12
26257    2015-12
26258    2016-12
26259    2017-12
26260    2018-12
26261    2019-12
26262    2020-12
26263    2021-12
26264    2022-12
Name: YearMonth, dtype: object

In [78]:
filtered_hist

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
54,2009-12-01,7.036530,7.427448,6.879077,6.960518,291100.0,0.0,0,2009
67,2010-12-01,9.468220,10.606312,9.247751,10.421596,376800.0,0.0,0,2010
80,2011-12-01,8.046126,8.883735,7.754232,8.667987,189600.0,0.0,0,2011
95,2012-12-01,9.289784,9.644898,9.055409,9.637795,382700.0,0.0,0,2012
111,2013-12-01,14.829822,16.288980,14.532034,16.110308,619500.0,0.0,0,2013
127,2014-12-01,16.771265,18.213455,16.422349,17.089169,369400.0,0.0,0,2014
143,2015-12-01,16.241933,16.588192,14.301276,15.243421,310200.0,0.0,0,2015
159,2016-12-01,20.950010,22.458410,20.589669,22.207010,477700.0,0.0,0,2016
174,2017-12-01,30.719469,30.874793,28.821078,29.511402,788300.0,0.0,0,2017
190,2018-12-01,29.569555,31.239446,26.096530,28.477018,688900.0,0.0,0,2018


In [79]:
# Format to datetime
filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])
hist['Date'] = pd.to_datetime(hist['Date'])
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year > earliest_date.year]
# Find the earliest date in the filtered_hist dataframe
earliest_date = filtered_hist['Date'].min()
# Filter 'hist' dataframe for rows with a date greater than the earliest date
filtered_hist_after_earliest = hist[hist['Date'].dt.year < earliest_date.year]

C:\Users\germa\AppData\Local\Temp/ipykernel_6404/2001008434.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hist['Date'] = pd.to_datetime(filtered_hist['Date'])


In [80]:
# Filter the resulting dataframe for rows with the same month as the earliest date
appended_hist = filtered_hist_after_earliest[filtered_hist_after_earliest['Date'].dt.month == earliest_date.month]

In [81]:
# Concatenate the filtered_hist dataframe with the appended_hist dataframe
prices_filtered = pd.concat([filtered_hist, appended_hist]).sort_values(by='year').reset_index(drop=True)

In [82]:
prices_filtered['ticker'] = ticker
prices_filtered['Growth -1'] = (prices_filtered['Close'] - prices_filtered['Close'].shift(periods=1)) /  prices_filtered['Close'].shift(periods=1)
prices_filtered['Growth +1'] = (prices_filtered['Close'].shift(periods=-1) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['Growth +5'] = (prices_filtered['Close'].shift(periods=-5) - prices_filtered['Close']) /  prices_filtered['Close']
prices_filtered['avgGrowth -10'] = prices_filtered['Growth -1'].rolling(min_periods=10, window=10).sum() / 10
prices_filtered['avgGrowth -5'] = prices_filtered['Growth -1'].rolling(min_periods=5, window=5).sum() / 5

# filter_years = pd.to_datetime(ticker_file['Date']).dt.year.tolist()
# histgrouped_filtered = prices_filtered[prices_filtered['year'].isin(filter_years)]
prices_filtered.reset_index(names=['longevity'], inplace=True)

In [83]:
prices_filtered.head(15)

,longevity,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,ticker,Growth -1,Growth +1,Growth +5,avgGrowth -10,avgGrowth -5
0,0,2005-12-01,6.559063,7.807974,6.472459,7.270122,2252900.0,0.0,0,2005,ITRN,NaN,-0.046736,0.433483,NaN,NaN
1,1,2006-12-01,7.349386,7.703961,6.704703,6.930342,912300.0,0.0,0,2006,ITRN,-0.046736,-0.253829,0.250730,NaN,NaN
2,2,2007-12-01,4.806851,5.231947,4.648024,5.171219,859900.0,0.0,0,2007,ITRN,-0.253829,-0.250547,0.863738,NaN,NaN
3,3,2008-12-01,4.092956,4.739770,3.790756,3.875584,1223100.0,0.0,0,2008,ITRN,-0.250547,0.795992,3.156872,NaN,NaN
4,4,2009-12-01,7.036530,7.427448,6.879077,6.960518,291100.0,0.0,0,2009,ITRN,0.795992,0.497244,1.455158,NaN,NaN
5,5,2010-12-01,9.468220,10.606312,9.247751,10.421596,376800.0,0.0,0,2010,ITRN,0.497244,-0.168267,0.462676,NaN,0.148425
6,6,2011-12-01,8.046126,8.883735,7.754232,8.667987,189600.0,0.0,0,2011,ITRN,-0.168267,0.111884,1.561957,NaN,0.124119
7,7,2012-12-01,9.289784,9.644898,9.055409,9.637795,382700.0,0.0,0,2012,ITRN,0.111884,0.671576,2.062049,NaN,0.197261
8,8,2013-12-01,14.829822,16.288980,14.532034,16.110308,619500.0,0.0,0,2013,ITRN,0.671576,0.060760,0.767627,NaN,0.381686
9,9,2014-12-01,16.771265,18.213455,16.422349,17.089169,369400.0,0.0,0,2014,ITRN,0.060760,-0.108007,0.337786,NaN,0.234639
